# Housing Prices

## Imports

In [1]:
import datacleaning as dc
import linregvis as lrv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split 
from sklearn.dummy import DummyRegressor
from statsmodels.graphics.regressionplots import abline_plot
%matplotlib inline
sns.set_style('darkgrid')

## Getting the Data

In [2]:
df = dc.test_split(
    dc.get_data('data/kc_house_data.csv')
)
X_train = dc.clean_it(df[0])
X_test = dc.clean_it(df[1])
y_train = df[2]
y_test = df[3]

## Model #1

In [6]:
formula = "price ~ sqft_living"
model_1 = lrv.run_lr(formula, X_train, y_train)
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     7571.
Date:                Mon, 19 Oct 2020   Prob (F-statistic):               0.00
Time:                        17:02:27   Log-Likelihood:            -2.3887e+05
No. Observations:               17277   AIC:                         4.777e+05
Df Residuals:                   17273   BIC:                         4.778e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -5.894e+05   1.45e+04    -40.648      0.000   -6.18e+05   -5.61e+05
sqft_living   177.4785      3.121     56.868      0.000     171.361     183.596
grade        9.859e+04   2442.422     40.366      0.000    9.38e+04    1.03e+05
waterfront   8.969e+05   2.34e+04     38.300      0.000    8.51e+05    9.43e+05
==============================================================================
Omnibus:                    12971.258   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           791353.950
Skew:                           3.058   Prob(JB):                         0.00
Kurtosis:                      35.586   Cond. No.                     2.87e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.87e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Visualization

In [ ]:
fig, ax = plt.subplots(figsize = (14,6))
sns.regplot(x = X_train.sqft_living, y = y_train, ax=ax);

### Validation

## Model #2

In [ ]:
formula = "price ~ sqft_living + condition_2 + condition_3 + condition_4 + condition_5"
model_2 = lrv.run_lr(formula, X_train, y_train)
model_2.summary()

### Validation

## Model #3

### Validation